In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split, explode,substring, length, udf
from pyspark.sql.types import DecimalType, StringType
from itertools import cycle
from pyspark.ml.regression import LinearRegression
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

from pyspark.sql import functions as F
from pyspark.sql import types as T

In [ ]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [ ]:
# spark is an custom SparkSession based on some config to work with Jupyter notebooks
iv = spark.read.csv("hdfs://localhost:9000/user/lavish/data/investments.csv"
                , header='true'
                , inferSchema='true')

In [ ]:
startYear=1995
endYear=2015

In [ ]:
type(iv)

In [ ]:
iv.schema.names

In [ ]:
filteredIV = iv.filter(iv.raised_amount_usd.isNotNull())

In [ ]:
from pyspark.sql.types import DoubleType
splittedCategoryIV = filteredIV.select('raised_amount_usd',  substring('funded_at',-4,4).cast('int').alias('year')
                       , split(col("company_category_list")
                       , "[|]s*").alias("categoryArr"))

In [ ]:
explodedIV=splittedCategoryIV.select('raised_amount_usd','year', explode('categoryArr').alias('category'))

In [ ]:
explodedIV.show(10)

In [ ]:
explodedIV.createOrReplaceTempView("investments")

In [ ]:
sqlDF = spark.sql("SELECT * FROM investments")
sqlDF.show(5)

### Year Wise

In [ ]:
SQLQUERY =  """
            SELECT CATEGORY, 
            CAST(YEAR AS INT), 
            SUM(RAISED_AMOUNT_USD) AS TOTAL, 
            CAST(SUM(RAISED_AMOUNT_USD) AS DECIMAL(30)) AS TOTAL_DEC 
            FROM INVESTMENTS GROUP 
            BY CATEGORY, YEAR 
            """
#  ORDER BY YEAR DESC, TOTAL DESC

In [ ]:
sqlDF = spark.sql(SQLQUERY)
sqlDF.show(5)

In [ ]:
from pyspark.ml.feature import VectorAssembler

vectorAssembler = VectorAssembler(inputCols = ['YEAR'], outputCol = 'FEATURES')
featureDF = vectorAssembler.transform(sqlDF).select('CATEGORY', 'FEATURES', 'TOTAL')

featureDF.show(5)

In [ ]:
f = featureDF.select('CATEGORY').distinct()


In [ ]:
topCategories = [row.CATEGORY for row in f.collect()]


In [ ]:
len(topCategories)

In [ ]:
# Features matrix to predict the amount for the Year 2020

l =  [(2020,)]

rdd = sc.parallelize(l)
test = rdd.map(lambda x: Row(YEAR=x[0] ))
testDF = sqlContext.createDataFrame(test)

vectorAssembler = VectorAssembler(inputCols = ['YEAR'], outputCol = 'FEATURES')
vectorDF = vectorAssembler.transform(testDF).select('FEATURES')

In [164]:
#count = 0 
statInfo = {}
for category in topCategories:
    #print(category)
    categoryDF=featureDF.filter(featureDF.CATEGORY == category)
    if(categoryDF.count() > 10):
        #count +=1
        lr = LinearRegression(featuresCol = 'FEATURES', labelCol='TOTAL', maxIter=10, regParam=0.3, elasticNetParam=0.8)
        lr_model = lr.fit(categoryDF)
        if (lr_model.summary.r2 >= .5):
            statInfo[category] = { 'Gradient' : str(lr_model.coefficients),
                                   'Intercept' : str(lr_model.intercept),
                                   'RMSE' : str(lr_model.summary.rootMeanSquaredError),
                                   'R2' : str(lr_model.summary.r2),
                                   'Pediction' : lr_model.transform(vectorDF).take(1)[0].prediction                               }

print (statInfo)

{'Personal Finance': {'Gradient': '[309531844.5424496]', 'Intercept': '-620846579526.5864', 'RMSE': '1140333317.135928', 'R2': '0.5257188832329349', 'Pediction': 4407746449.161743}, 'Reviews and Recommendations': {'Gradient': '[93843505.10535592]', 'Intercept': '-188076601008.07297', 'RMSE': '225260027.9853369', 'R2': '0.6748252913534014', 'Pediction': 1487279304.7460022}, 'Health Care': {'Gradient': '[1089864218.1450346]', 'Intercept': '-2179250886376.2097', 'RMSE': '2023745521.7204287', 'R2': '0.8747611056595263', 'Pediction': 22274834276.76001}, 'Application Performance Monitoring': {'Gradient': '[146386390.37091857]', 'Intercept': '-293690139320.35425', 'RMSE': '470405923.3819028', 'R2': '0.566899821885139', 'Pediction': 2010369228.901245}, 'Credit Cards': {'Gradient': '[63341922.86089984]', 'Intercept': '-127009681219.96451', 'RMSE': '259743121.04301742', 'R2': '0.5027780036421106', 'Pediction': 941002959.0531769}, 'Collaboration': {'Gradient': '[288729181.3048054]', 'Intercept': 

In [166]:
statInfo

{'Personal Finance': {'Gradient': '[309531844.5424496]',
  'Intercept': '-620846579526.5864',
  'RMSE': '1140333317.135928',
  'R2': '0.5257188832329349',
  'Pediction': 4407746449.161743},
 'Reviews and Recommendations': {'Gradient': '[93843505.10535592]',
  'Intercept': '-188076601008.07297',
  'RMSE': '225260027.9853369',
  'R2': '0.6748252913534014',
  'Pediction': 1487279304.7460022},
 'Health Care': {'Gradient': '[1089864218.1450346]',
  'Intercept': '-2179250886376.2097',
  'RMSE': '2023745521.7204287',
  'R2': '0.8747611056595263',
  'Pediction': 22274834276.76001},
 'Application Performance Monitoring': {'Gradient': '[146386390.37091857]',
  'Intercept': '-293690139320.35425',
  'RMSE': '470405923.3819028',
  'R2': '0.566899821885139',
  'Pediction': 2010369228.901245},
 'Credit Cards': {'Gradient': '[63341922.86089984]',
  'Intercept': '-127009681219.96451',
  'RMSE': '259743121.04301742',
  'R2': '0.5027780036421106',
  'Pediction': 941002959.0531769},
 'Collaboration': {'Gr